##Importations
- codecs pour les encodages
- pandas et numpy pour les calculs sur tableaux
- matplotlib pour les graphiques
- itertools pour les itérateurs sophistiqués (paires sur liste, ...)

In [6]:
# -*- coding: utf8 -*-
import codecs
import features
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools as it
import pickle
import networkx as nx
#%pylab inline
#pd.options.display.mpl_style = 'default'
debug=False
from __future__ import print_function

In [7]:
import datetime
def dateheure():
    return datetime.datetime.utcnow().strftime('%y%m%d%H%M')

In [8]:
saut="\n"

###Préparation des matrices de traits

In [9]:
features.add_config('bdlexique.ini')
fs=features.FeatureSystem('phonemes')

In [10]:
samplePrefix="MGC-150815-extend-01-X-paradigmes"
sampleFile=samplePrefix+".csv"
goldFile="MGC-150815-total-1508150555-paradigmes.csv"
analysisPrefix="MGC-150815-extend-01-X"
logfile_name=analysisPrefix+samplePrefix+".log"
logfile = codecs.open(logfile_name,mode='w',encoding="utf8")

###Préparation des cases du paradigme

In [11]:
casesPrincipales= [
        'inf', 'pi1S', 'pi2S', 'pi3S', 'pi1P', 'pi2P', 'pi3P', 'ii1S',
        'ii2S', 'ii3S', 'ii1P', 'ii2P', 'ii3P', 
        'fi1S', 'fi2S', 'fi3S', 'fi1P', 'fi2P',
        'fi3P', 'pI2S', 'pI1P', 'pI2P', 'ps1S', 'ps2S', 'ps3S', 'ps1P',
        'ps2P', 'ps3P', 
        'pc1S', 'pc2S', 'pc3S', 'pc1P', 'pc2P', 'pc3P', 'pP',
        'ppMS', 'ppMP', 'ppFS', 'ppFP'
            ]
casesSecondaires= [
       'ai1S', 'ai2S', 'ai3S', 'ai1P', 'ai2P', 'ai3P', 'is1S', 'is2S', 'is3S', 'is1P', 'is2P', 'is3P'
            ]
casesTotales=casesPrincipales+casesSecondaires

#Préparation du calcul des analogies

###Calcul de la différence entre deux formes

In [12]:
def diff(mot1,mot2):
    result=[]
    diff1=""
    diff2=""
    same=""
    vide="."
    lmax=max(len(mot1),len(mot2))
    lmin=min(len(mot1),len(mot2))
    for index in range(lmax):
        if index < lmin:
            if mot1[index]!=mot2[index]:
                diff1+=mot1[index]
                diff2+=mot2[index]
                same+=vide
            else:
                same+=mot1[index]
                diff1+=vide
                diff2+=vide
        elif index < len(mot1):
            diff1+=mot1[index]
        elif index < len(mot2):
            diff2+=mot2[index]
    diff1=diff1.lstrip(".")
    diff2=diff2.lstrip(".")
#    return (same,diff1,diff2,diff1+"_"+diff2)
    return (diff1+"-"+diff2)

###Accumulation des paires appartenant à un patron

In [13]:
def rowDiff(row, patrons):
    result=diff(row[0],row[1])
    if not result in patrons:
        patrons[result]=(formesPatron(),formesPatron())
    patrons[result][0].ajouterFormes(row[0])
    patrons[result][1].ajouterFormes(row[1])
    return (result[0],result[1])

###Transformation d'un patron en RegExp

In [14]:
def patron2regexp(morceaux):
    result="^"
    for morceau in morceaux:
        if morceau=="*":
            result+="(.*)"
        elif len(morceau)>1:
            result+="(["+morceau+"])"
        else:
            result+=morceau
    result+="$"
    result=result.replace(")(","")
    return result

###Substitution de sortie 
???

In [15]:
def remplacementSortie(sortie):
    n=1
    nsortie=""
    for lettre in sortie:
        if lettre==".":
            nsortie+="\g<%d>"%n
            n+=1
        else:
            nsortie+=lettre
    return nsortie

#Classe pour la gestion des patrons, des classes et des transformations

In [16]:
class paireClasses:
    def __init__(self,case1,case2):
        self.case1=case1
        self.case2=case2
        self.nom=case1+"-"+case2
        self.classes1=classesPaire(case1,case2)
        self.classes2=classesPaire(case2,case1)

    def ajouterPatron(self,n,patron,motif):
        if n==1:
            self.classes1.ajouterPatron(patron,motif)
        elif n==2:
            self.classes2.ajouterPatron(patron,motif)
        else:
            print ("le numéro de forme n'est pas dans [1,2]",n,file=logfile)

    def ajouterPaire(self,forme1,forme2):
        self.classes1.ajouterPaire(forme1,forme2)
        self.classes2.ajouterPaire(forme2,forme1)
        
    def calculerClasses(self):
        return(self.classes1,self.classes2)

    
class classesPaire:
    '''
    Gestion des patrons, des classes et des transformations
    
    ajouterPatron : ajoute un patron et son motif associé (MGL)
    ajouterPaire : ajoute une paire de formes, calcule la classe de la forme1 et la règle sélectionnée
    sortirForme : cacule les formes de sortie correspondant à la forme1 avec leurs coefficients respectifs
    '''
    def __init__(self,case1,case2):
        self.case1=case1
        self.case2=case2
        self.nom=case1+"-"+case2
        self.classe={}
        self.nbClasse={}
        self.patrons={}
        self.entree={}
        self.sortie={}
    
    def ajouterPatron(self,patron,motif):
        self.patrons[patron]=motif
        (entree,sortie)=patron.split("-")
        self.entree[patron]=entree.replace(u".",u"(.)")
        self.sortie[patron]=remplacementSortie(sortie)
    
    def ajouterPaire(self,forme1,forme2):
        '''
        on calcule la classe de la paire idClasseForme et la règle sélectionnée
        on incrémente le compteur de la classe et celui de la règle sélectionnée à l'intérieur de la classe
        '''
        classeForme=[]
        regleForme=""
        for patron in self.patrons:
            if re.match(self.patrons[patron],forme1):
                classeForme.append(patron)
                '''
                le +"$" permet de forcer l'alignement à droite pour les transformations suffixales
                '''
                if forme2==re.sub(self.entree[patron]+"$",self.sortie[patron],forme1):
                    regleForme=patron
        idClasseForme=", ".join(classeForme)
        if not idClasseForme in self.classe:
            self.classe[idClasseForme]={}
            self.nbClasse[idClasseForme]=0
        if not regleForme in self.classe[idClasseForme]:
            self.classe[idClasseForme][regleForme]=0
        self.nbClasse[idClasseForme]+=1
        self.classe[idClasseForme][regleForme]+=1

    def sortirForme(self,forme):
        classeForme=[]
        sortieForme={}
        for patron in self.patrons:
            if re.match(self.patrons[patron],forme):
                classeForme.append(patron)
        if classeForme:
            idClasseForme=", ".join(classeForme)
            if idClasseForme in self.nbClasse:
                nTotal=self.nbClasse[idClasseForme]
                for patron in self.classe[idClasseForme]:
                    sortie=re.sub(self.entree[patron]+"$",self.sortie[patron],forme)
                    sortieForme[sortie]=float(self.classe[idClasseForme][patron])/nTotal
            else:
                if debug:
                    print (forme, file=logfile)
                    print ("pas de classe",idClasseForme, file=logfile)
                    print ("%.2f par forme de sortie" % (float(1)/len(classeForme)), file=logfile)
                nTotal=len(classeForme)
                for patron in classeForme:
                    sortie=re.sub(self.entree[patron]+"$",self.sortie[patron],forme)
                    sortieForme[sortie]=float(1)/nTotal
        else:
            print (forme, file=logfile) 
            print ("pas de patron", file=logfile)
        return sortieForme
        

##Appliquer la formule de calcul des différences entre chaines à chaque ligne

>si il y a au moins une ligne

>>on applique la différence à la ligne

>>on calcule les deux patrons par suppression des points initiaux

>>on renvoie le groupement par patrons (1&2)

>sinon

>>on renvoie le paradigme vide d'origine

In [17]:
def OLDrapports(paradigme):
    (case1,case2,lexeme)= paradigme.columns.values.tolist()
    patrons=pairePatrons(case1,case2)
    if len(paradigme)>0:
#        for index, row in paradigme.iterrows():
#            patrons.ajouterFormes(row[0],row[1],diff(row[0],row[1]))
        paradigme.apply(lambda x: patrons.ajouterFormes(x[case1],x[case2],diff(x[case1],x[case2])), axis=1)
        (regles1,regles2)=patrons.calculerGM()
    return patrons.calculerGM()

In [18]:
def rapports(paradigme):
    if len(paradigme.columns.values.tolist())==2:
        (case1,lexeme)= paradigme.columns.values.tolist()
        case2=case1
    else:
        (case1,case2,lexeme)= paradigme.columns.values.tolist()
    patrons=pairePatrons(case1,case2)
    classes=paireClasses(case1,case2)
    if len(paradigme)>0:
        paradigme.apply(lambda x: patrons.ajouterFormes(x[case1],x[case2],diff(x[case1],x[case2])), axis=1)
        (regles1,regles2)=patrons.calculerGM()
        for regle in regles1:
            classes.ajouterPatron(1,regle,regles1[regle])
        for regle in regles2:
            classes.ajouterPatron(2,regle,regles2[regle])
        paradigme.apply(lambda x: classes.ajouterPaire(x[case1],x[case2]), axis=1)
    (classes1,classes2)=classes.calculerClasses()
    return (classes1,classes2)

###Dédoubler les lignes avec des surabondances dans *colonne*
>identifier une ligne avec surabondance

>>ajouter les lignes correspondant à chaque valeur

>>ajouter le numéro de la ligne initiale dans les lignes à supprimer

>supprimer les lignes avec surabondance

NB : il faut préparer le tableau pour avoir une indexation qui permette l'ajout des valeurs individuelles et la suppression des lignes de surabondances

In [19]:
def splitCellMates(df,colonne):
    '''
    Calcul d'une dataframe sans surabondance par dédoublement des valeurs
    '''
    test=df.reset_index()
    del test["index"]
    splitIndexes=[]
    for index,ligne in test.iterrows():
        if "," in ligne[colonne]:
            valeurs=set(ligne[colonne].split(","))
            nouvelleLigne=ligne
            for valeur in valeurs:
                nouvelleLigne[colonne]=valeur
                test=test.append(nouvelleLigne,ignore_index=True)
            splitIndexes.append(index)
    if splitIndexes:
        test=test.drop(test.index[splitIndexes])
    return test


In [20]:
paradigmes=pd.read_csv(sampleFile,sep=";",encoding="utf8")
del paradigmes[u"Unnamed: 0"]

In [21]:
phonologicalMap=analysisPrefix[-2:]
if debug: print(phonologicalMap)
neutralisationsNORD=(u"6û",u"9ê")
neutralisationsSUD=(u"e2o",u"E9O")
if phonologicalMap=="-N":
    neutralisations=neutralisationsNORD
elif phonologicalMap=="-S":
    neutralisations=neutralisationsSUD
else:
    neutralisations=(u"",u"")
    phonologicalMap=("-X")
bdlexiqueIn = unicode(u"èò"+neutralisations[0])
bdlexiqueNum = [ord(char) for char in bdlexiqueIn]
neutreOut = unicode(u"EO"+neutralisations[1])
neutralise = dict(zip(bdlexiqueNum, neutreOut))

In [22]:
def recoder(chaine,table=neutralise):
    if type(chaine)==str:
        temp=unicode(chaine.decode('utf8')).translate(table)
        result=temp.encode('utf8')
    elif type(chaine)==unicode:
        result=chaine.translate(table)
    else:
        result=chaine
    return result

In [23]:
GOLD=pd.read_csv(goldFile,sep=";",encoding="utf8")
del GOLD[u"Unnamed: 0"]
goldCases=GOLD.columns.tolist()
goldCases.remove(u"lexeme")
for case in goldCases:
    GOLD[case]=GOLD[case].apply(lambda x: recoder(x))
    
paradigmesGOLD=GOLD[paradigmes.columns.values.tolist()]
paradigmesGOLD

,lexeme,ai1P,ai1S,ai2P,ai2S,ai3P,ai3S,fi1P,fi1S,fi2P,...,ppFP,ppFS,ppMP,ppMS,ps1P,ps1S,ps2P,ps2S,ps3P,ps3S
0,abaisser,abEsam,abEsE,abEsat,abEsa,abEsEr,abEsa,abEs6rô,abEs6rE,abEs6re,...,abEse,abEse,abEse,abEse,abEsjô,abEs,abEsje,abEs,abEs,abEs
1,abandonner,abâdOnam,abâdOnE,abâdOnat,abâdOna,abâdOnEr,abâdOna,abâdOn6rô,abâdOn6rE,abâdOn6re,...,abâdOne,abâdOne,abâdOne,abâdOne,abâdOnjô,abâdOn,abâdOnje,abâdOn,abâdOn,abâdOn
2,abasourdir,abazurdim,abazurdi,abazurdit,abazurdi,abazurdir,abazurdi,abazurdirô,abazurdirE,abazurdire,...,abazurdi,abazurdi,abazurdi,abazurdi,abazurdisjô,abazurdis,abazurdisje,abazurdis,abazurdis,abazurdis
3,abattre,abatim,abati,abatit,abati,abatir,abati,abatrô,abatrE,abatre,...,abaty,abaty,abaty,abaty,abatjô,abat,abatje,abat,abat,abat
4,abcéder,absEdam,absEdE,absEdat,absEda,absEdEr,absEda,absEd6rô,absEd6rE,absEd6re,...,NaN,NaN,NaN,absEde,absEdjô,absEd,absEdje,absEd,absEd,absEd
5,abdiquer,abdikam,abdikE,abdikat,abdika,abdikEr,abdika,abdik6rô,abdik6rE,abdik6re,...,abdike,abdike,abdike,abdike,abdikjô,abdik,abdikje,abdik,abdik,abdik
6,abecquer,abEkam,abEkE,abEkat,abEka,abEkEr,abEka,abEk6rô,abEk6rE,abEk6re,...,abEke,abEke,abEke,abEke,abEkjô,abEk,abEkje,abEk,abEk,abEk
7,aberrer,abEram,abErE,abErat,abEra,abErEr,abEra,abEr6rô,abEr6rE,abEr6re,...,NaN,NaN,NaN,abEre,abErjô,abEr,abErje,abEr,abEr,abEr
8,abhorrer,abOram,abOrE,abOrat,abOra,abOrEr,abOra,abOr6rô,abOr6rE,abOr6re,...,abOre,abOre,abOre,abOre,abOrjô,abOr,abOrje,abOr,abOr,abOr
9,abjurer,abZyram,abZyrE,abZyrat,abZyra,abZyrEr,abZyra,abZyr6rô,abZyr6rE,abZyr6re,...,abZyre,abZyre,abZyre,abZyre,abZyrjô,abZyr,abZyrje,abZyr,abZyr,abZyr


- sampleCases pour la liste des cases effectivement représentées dans le corpus de départ 

In [24]:
sampleCases=paradigmes.columns.values.tolist()
sampleCases.remove(u"lexeme")

In [25]:
paradigmes.stack().value_counts(dropna=True).sum()

17784

In [26]:
with open(analysisPrefix+'-Regles.pkl', 'rb') as input:
    resultatsLecture = pickle.load(input)

###Comparer les cases analysées avec l'ensemble de toutes les cases

In [27]:
analyseCases=list(set([case for (case,autre) in resultatsLecture.keys()]))
if sorted(analyseCases)!=sorted(casesTotales):
    print ("Attention l'analyse ne comprend pas toutes les cases")

Attention l'analyse ne comprend pas toutes les cases


In [28]:
class paradigmeDistribution:
    '''
    Gestion des distributions dans les cases du paradigme
    '''

    def __init__(self,lexeme):
        self.lexeme=lexeme
        self.formes={i:{} for i in analyseCases}

    def ajouterFormes(self,case,formes,coef=1.0):
        for forme in formes:
            if not forme in self.formes[case]:
                self.formes[case][forme]=0
            self.formes[case][forme]+=formes[forme]*coef
            
    def normaliserDistributions(self,caseListe=analyseCases):
        normalesDistributions={i:{} for i in caseListe}
        for case in caseListe:
            total=0
            for element in self.formes[case]:
                total+=self.formes[case][element]
            for element in self.formes[case]:
                normalesDistributions[case][element]=float(self.formes[case][element])/total
        return normalesDistributions
        

In [29]:
def generateForms(lexeme):
    candidats=paradigmeDistribution(lexeme)
    casesSamples=paradigmes[paradigmes["lexeme"]==lexeme].columns[paradigmes[paradigmes["lexeme"]==lexeme].notnull().iloc[0]].tolist()
    casesSamples.remove("lexeme")
    for caseDepart in casesSamples:
        formeDepart=paradigmes[paradigmes["lexeme"]==lexeme][caseDepart].iloc[0]
        if debug: print (caseDepart,formeDepart, file=logfile)
#        if formeDepart!="nan":
        for case in analyseCases:
            if debug: print (case, file=logfile)
            if not isinstance(resultatsLecture[(caseDepart, case)],str):
                if "," in formeDepart:
                    formesDepart=formeDepart.split(",")
                    coef=1.0/len(formesDepart)
                    for element in formesDepart:
                        candidats.ajouterFormes(case,resultatsLecture[(caseDepart, case)].sortirForme(element),coef)
                else:
                    candidats.ajouterFormes(case,resultatsLecture[(caseDepart, case)].sortirForme(formeDepart))
            else: 
                if debug: print ("str", resultatsLecture[(caseDepart, case)], file=logfile)
    return candidats

In [30]:
paradigmes[paradigmes["lexeme"]=="dormir"].columns[paradigmes[paradigmes["lexeme"]=="dormir"].notnull().iloc[0]].tolist()

[u'lexeme',
 u'ai1S',
 u'fi1S',
 u'fi2P',
 u'fi3P',
 u'fi3S',
 u'ii1S',
 u'ii2P',
 u'ii2S',
 u'ii3S',
 u'inf',
 u'pI2P',
 u'pI2S',
 u'pP',
 u'pi1S',
 u'pi2P',
 u'pi2S',
 u'pi3S',
 u'ps1S',
 u'ps3P']

In [31]:
def ajouterPoint(lexeme,forme,case):
    pointName="%s-%s-%s"%(lexeme,forme,case)
#    if not pointName in digraphe.nodes():
    tam=case[:2]
    if tam=="in": tam="inf"
    digraphe.add_node(pointName, tam='"%s"'%tam)
    graphe.add_node(pointName, tam='"%s"'%tam)
    return pointName

def ajouterFleche(pointDepart,pointSortie,coef):
    digraphe.add_edge(pointDepart,pointSortie,weight=float(coef))
    if digraphe.has_edge(pointSortie,pointDepart):
        coefGraphe=float(digraphe.edge[pointSortie][pointDepart]["weight"]+coef)/2
        graphe.add_edge(pointDepart,pointSortie,weight=coefGraphe)

In [32]:
def generateParadigms(generation1,genDigraphe=True):
    lexeme=generation1.lexeme
    distributionInitiale=generation1.normaliserDistributions()
    candidats=paradigmeDistribution(lexeme)
    for caseDepart in analyseCases:
        for formeDepart in distributionInitiale[caseDepart]:
            if formeDepart:
                pointDepart=ajouterPoint(lexeme,formeDepart,caseDepart)
                coefDepart=distributionInitiale[caseDepart][formeDepart]
                if debug: print (caseDepart,formeDepart, file=logfile)
                for caseSortie in analyseCases:
                    distributionSortieBrute=resultatsLecture[(caseDepart, caseSortie)].sortirForme(formeDepart)
                    if distributionSortie:
                        if not genDigraphe:
                            distributionSortie={f:distributionSortieBrute[f] for f in distributionSortieBrute if distributionSortieBrute[f] in distributionInitiale[caseSortie]}
                        else:
                            distributionSortie=distributionSortieBrute
                        if debug: print (caseSortie,distributionSortie,distributionInitiale[caseDepart], file=logfile)
                        candidats.ajouterFormes(caseSortie,distributionSortie,distributionInitiale[caseDepart][formeDepart])
                        for formeSortie in distributionSortie:
                            pointSortie=ajouterPoint(lexeme,formeSortie,caseSortie)
                            coefSortie=distributionSortie[formeSortie]
                            ajouterFleche(pointDepart,pointSortie,float(coefDepart*coefSortie))
    return candidats

In [36]:
def generate(lexeme):
#    print (lexeme,end=", ")
    generation1=generateForms(lexeme)
#    print ("génération 2",end=", ")
    generation2=generateParadigms(generation1)
#    print ("génération 3")
    return generation2

In [37]:
len(paradigmes.dropna(thresh=6)["lexeme"])

1016

In [41]:
%%time
debug=False
listeTest=[u"manger",u"boire",u"dormir",u"aller",u"avoir"]
#listeTest=paradigmes.dropna(thresh=3)["lexeme"].values.tolist()
nbVerbes=len(listeTest)
print (nbVerbes)
digraphe=nx.DiGraph()
graphe=nx.Graph()
for i,element in enumerate(listeTest):
    if (i%10)==0: print (i, dateheure(), int(100*float(i)/nbVerbes), end=", ")
    generate(element,False)

0 1508181112, 10 1508181113, 20 1508181113, 30 1508181113, 40 1508181114, 50 1508181114, 60 1508181115, 70 1508181115, 80 1508181115, 90 1508181116, 100 1508181116, 110 1508181117, 120 1508181117, 130 1508181118, 140 1508181118, 150 1508181119, 160 1508181119, 170 1508181119, 180 1508181120, 190 1508181120, 200 1508181121, 210 1508181121, 220 1508181121, 230 1508181122, 240 1508181122, 250 1508181123, 260 1508181123, 270 1508181123, 280 1508181124, 290 1508181124, 300 1508181124, 310 1508181125, 320 1508181125, 330 1508181125, 340 1508181126, 350 1508181126, 360 1508181127, 370 1508181127, 380 1508181128, 390 1508181128, 400 1508181128, 410 1508181129, 420 1508181129, 430 1508181130, 440 1508181130, 450 1508181131, 460 1508181131, 470 1508181131, 480 1508181132, 490 1508190145, 500 1508190145, 510 1508190146, 520 1508190146, 530 1508190146, 540 1508190147, 550 1508190147, 560 1508190147, 570 1508190148, 580 1508190148, 590 1508190149, 600 1508190149, 610 1508190149, 620 1508190150, 630

In [42]:
%%time
versionStamp=dateheure()
nx.write_dot(digraphe,u"digraphe-%s.dot"%versionStamp)
nx.write_dot(graphe,u"graphe-%s.dot"%versionStamp)

CPU times: user 8min 11s, sys: 32.7 s, total: 8min 44s
Wall time: 8min 59s


In [5]:
#graphe=nx.Graph()
#graphe=nx.read_dot(u"graphe-1508180522.dot")

In [43]:
%%time
cliques=list(nx.algorithms.clique.find_cliques(graphe))

CPU times: user 1min 17s, sys: 3min 33s, total: 4min 51s
Wall time: 8min 16s


In [44]:
for clique in sorted(cliques,key=lambda x: len(x),reverse=True)[:50]:
    print (len(clique),end=", ")
    print (clique)

45, [u'faire-f6z\xe2-pP', u'faire-f6z\xf4-pi1P', u'faire-fE-pI2S', u'faire-fE-pi2S', u'faire-fE-pi1S', u'faire-fE-pi3S', u'faire-f6rE-pc3P', u'faire-f6rE-pc3S', u'faire-f6rE-pc2S', u'faire-fEt-pi2P', u'faire-fi-is3S', u'faire-fas-ps3S', u'faire-fEr-inf', u'faire-f6rE-pc1S', u'faire-fas-ps3P', u'faire-fas-ps1S', u'faire-f6re-fi2P', u'faire-fE-ppMS', u'faire-fE-ppMP', u'faire-fi-ai3S', u'faire-fEt-pI2P', u'faire-f6ra-fi3S', u'faire-f6r\xf4-fi3P', u'faire-f6r\xf4-fi1P', u'faire-fir-ai3P', u'faire-fi-ai2S', u'faire-fasje-ps2P', u'faire-f6ra-fi2S', u'faire-fim-ai1P', u'faire-f6zE-ii2S', u'faire-f6zE-ii3P', u'faire-fEt-ppFS', u'faire-f\xf4-pi3P', u'faire-fEt-ppFP', u'faire-fas-ps2S', u'faire-f6rj\xf4-pc1P', u'faire-f6zj\xf4-ii1P', u'faire-f6z\xf4-pI1P', u'faire-f6zE-ii1S', u'faire-f6zje-ii2P', u'faire-f6rE-fi1S', u'faire-f6rje-pc2P', u'faire-fi-ai1S', u'faire-fasj\xf4-ps1P', u'faire-f6zE-ii3S']
44, [u'avoir-ys-is1S', u'avoir-a-pi2S', u'avoir-y-ppMS', u'avoir-ave-pi2P', u'avoir-E-ps2S', u'avo

In [ ]:
nombreElements={element:paradigmes[paradigmes["lexeme"]==element].stack().value_counts(dropna=True).sum()-1 for element in listeTest}
for element in sorted(nombreElements,key=nombreElements.get, reverse=True):
    print (element, nombreElements[element])

faire 45
avoir 44
devoir 41
dire 41
être 41
aller 40
venir 39
trouver 39
voir 37
passer 36
sortir 35
vouloir 35
pouvoir 35
partir 35
tirer 34
prendre 34
donner 34
aimer 34
croire 34
savoir 34
rester 34
retrouver 32
attendre 32
comprendre 32
arriver 32
arrêter 32
mettre 31
parler 31
laisser 31
revenir 31
connaître 30
entendre 30
oublier 30
montrer 30
demander 29
sentir 29
lire 29
appeler 28
essayer 28
chercher 28
devenir 27
vivre 27
poser 27
rentrer 27
finir 27
mourir 27
rendre 27
quitter 27
ouvrir 27
penser 26
descendre 26
tenir 26
commencer 26
continuer 26
changer 25
manquer 25
suivre 25
cacher 25
accepter 25
remettre 25
entrer 25
tuer 25
retourner 24
regarder 24
emmener 24
perdre 24
jeter 24
porter 24
répondre 24
jouer 24
monter 24
fermer 23
écouter 23
recevoir 23
manger 23
garder 23
apprendre 23
toucher 23
écrire 23
asseoir 22
rappeler 22
lever 22
tomber 22
tourner 22
raconter 22
reconnaître 22
pousser 22
aider 21
boire 21
abandonner 21
conduire 21
serrer 21
imaginer 21
revoir 21
pa

##Comparer la sortie des cliques avec le paradigme de départ

In [ ]:
%%time
seuilClique=3
#paradigmesCLIQUES=pd.DataFrame(columns=paradigmes.columns.values.tolist())
paradigmesListe=[]
for n,clique in enumerate(sorted(cliques,key=lambda x: len(x),reverse=True)):
#    if seuilClique==0:
#        seuilClique=len(clique)-15
    if len(clique)>seuilClique:
        paradigmeClique={}
        sampleOK=True
#        print (n, len(clique))
        point=clique[0].split("-")
        lPoint=len(point)
#        print (point,"-".join(point[0:len(point)-2]))
        if lPoint==3:
            lexeme=point[0]
        else:
            lexeme="-".join(point[0:len(point)-2])
        paradigmeClique["lexeme"]=lexeme
        if n%100==0: print (n,end=", ")
        casesLexeme=paradigmes[paradigmes["lexeme"]==lexeme].notnull().columns.tolist()
#        casesLexeme=paradigmes[paradigmes["lexeme"]==lexeme].columns[paradigmes[paradigmes["lexeme"]==lexeme].notnull().iloc[0]].tolist()
        casesLexeme.remove("lexeme")
        if casesLexeme:
            for element in casesLexeme:
#                print (lexeme,element, paradigmes[paradigmes["lexeme"]==lexeme][element])
                pointCase=u"%s-%s-%s"% (lexeme,paradigmes[paradigmes["lexeme"]==lexeme][element].iloc[0],element)
                if debug: print (pointCase, clique)
                if not pointCase in clique:
                    sampleOK=False
        else:
            sampleOK=False
        if sampleOK:
            for element in clique:
                (lexeme,forme,taminfo)=element.split("-")
                paradigmeClique[taminfo]=forme
                if taminfo in paradigmes.columns:
                    if not paradigmes[(paradigmes["lexeme"]==lexeme)][taminfo].isnull().item():
                        if paradigmes[(paradigmes["lexeme"]==lexeme)][taminfo].item()!=forme:
                            sampleOK=False
                            print (paradigmes[(paradigmes["lexeme"]==lexeme)][taminfo].item(),forme,taminfo,end=", ")
            paradigmesListe.append(paradigmeClique)
        else:
            if debug:
                print ()
                print (lexeme,clique)
                print ()
paradigmesCLIQUES=pd.DataFrame(paradigmesListe,columns=GOLD.columns.values.tolist())
print (seuilClique)

0, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200, 1300, 1400, 1500

In [ ]:
paradigmesCLIQUES

In [ ]:
#paradigmesCLIQUES
for element in listeTest:
    if paradigmesCLIQUES[paradigmesCLIQUES["lexeme"]==element].stack().value_counts(dropna=True).sum()>1:
        print (element,paradigmesCLIQUES[paradigmesCLIQUES["lexeme"]==element].stack().value_counts(dropna=True).sum()-1)


In [ ]:
GOLD.loc[GOLD["lexeme"]=="dormir"].stack()

In [ ]:
def compareGOLD(row):
#    if row["lexeme"]=="dodo": 
#        debug=True
#    else:
#        debug=False
    print (row["lexeme"],end=" : ")
    lexeme=row["lexeme"]
    if not lexeme in conjugaisons:
        conjugaisons[lexeme]={"diff":[],"miss":[],"over":[],"overmiss":[],"ok":[]}
    identiques=[]
    differents=[]
    missing=[]
    over=[]
    for case in analyseCases:
        if debug:    
            print (lexeme,case)
            print (GOLD[GOLD["lexeme"]==lexeme][case].item())
            print ((GOLD[GOLD["lexeme"]==lexeme][case].isnull().iloc[0]))
        if not case in row:
            if debug: print ("not case")
            missing.append(u"%s : Ø ≠ %s" % (case,GOLD[GOLD["lexeme"]==lexeme][case].item()))
            lexical=False
            if debug: print ("absence",case)
        elif case in row[row.isnull()] and not GOLD[GOLD["lexeme"]==lexeme][case].isnull().iloc[0]:
            if debug: print ("not case")
            missing.append(u"%s : Ø ≠ %s" % (case,GOLD[GOLD["lexeme"]==lexeme][case].item()))
            lexical=False
            if debug: print ("CLIQUES +NaN",case)
        elif not case in row[row.isnull()] and GOLD[GOLD["lexeme"]==lexeme][case].isnull().iloc[0]:
            if debug: print ("not case")
            over.append(u"%s : %s ≠ Ø" % (case,row[case]))
            lexical=False
            if debug: print ("GOLD -NaN",case)            
        elif (row[case]==GOLD[GOLD["lexeme"]==lexeme][case].item()):
            if debug: print ("egal")
            identiques.append(u"%s : %s" % (case,row[case]))
            if debug: print ("valeurs id")
        elif (case in row[row.isnull()]) and (GOLD[GOLD["lexeme"]==lexeme][case].isnull().iloc[0]):
            if debug: print ("NaNs")
            identiques.append(u"%s : %s" % (case,u"ØØØ"))
            if debug: print ("deux NaN")
        else:
            if debug: print ("else")
            differents.append(u"%s : %s ≠ %s" % (case, row[case],GOLD[GOLD["lexeme"]==lexeme][case].item()))
            lexical=False
            if debug: print (u"différence",case)
    if differents:
        conjugaisons[lexeme]["diff"].append(differents)
        if debug: print ("DIFF",len(differents),"=>", len(identiques), len(identiques)-nombreElements[lexeme], end=saut)
        if debug1: print (", ".join(differents))
    if missing and over:
        conjugaisons[lexeme]["overmiss"].append(over+missing)
        if debug: print ("OVERMISS", len(over), len(missing),"=>", len(identiques), len(identiques)-nombreElements[lexeme], end=saut)
        if debug1: 
            print (", ".join(missing))
            print (", ".join(over))
    elif missing:
        conjugaisons[lexeme]["miss"].append(missing)
        if debug: print ("MISS", len(missing),"=>", len(identiques), len(identiques)-nombreElements[lexeme], end=saut)
        if debug1: 
            print (", ".join(missing))
    elif over:
        conjugaisons[lexeme]["over"].append(over)
        if debug: print ("OVER", len(over),"=>", len(identiques), len(identiques)-nombreElements[lexeme], end=saut)
        if debug1: 
            print (", ".join(over))
    if identiques:
        conjugaisons[lexeme]["ok"].append(identiques)
        if debug: print ("OK", len(identiques), len(identiques)-nombreElements[lexeme],end=saut)
        if debug1: print (", ".join(identiques))
#    print ()
    return lexical

In [ ]:
%%time
debug=False
debug1=False
saut="\n"
conjugaisons={}
print (", ".join(sampleCases))
paradigmesCLIQUES.apply(compareGOLD,axis=1)

In [ ]:
pb=[i for i in conjugaisons if conjugaisons[i]["diff"]]
for element in pb:
    print (element)
    print ("DIFF",conjugaisons[element]["diff"])
    print ("MISS",conjugaisons[element]["miss"],end="\n\n")

In [ ]:
good=[i for i in conjugaisons if not conjugaisons[i]["diff"]]
for lexeme in good:
    print (lexeme)
    for ident in conjugaisons[lexeme]:
        for element in conjugaisons[lexeme][ident]:
            print (ident)
            print (", ".join(element))
    print ()

In [ ]:
paradigmes[paradigmes["lexeme"]==u"éprouver"].stack()

In [ ]:
GOLD[GOLD["lexeme"]==u"éprouver"]

In [ ]:
paradigmesCLIQUES[paradigmesCLIQUES["lexeme"]==u"éprouver"].stack()

In [ ]:
paradigmesGOLD

In [ ]:
paradigmesGOLD.loc[paradigmesGOLD["lexeme"]=="boire"]["inf"]